In [1]:
import os
import json

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error,roc_auc_score
import xgboost as xgb
import lightgbm as lgb
import utils
import numpy as np
import pandas as pd
from optimization_utils import is_categorical
from caafe.preprocessing import make_datasets_numeric
from sklearn.model_selection import StratifiedKFold, KFold
from caafe import data
import heapq
from sklearn.neural_network import MLPClassifier, MLPRegressor
from tabpfn import TabPFNRegressor, TabPFNClassifier

######
from autogluon.tabular import TabularPredictor

In [ ]:
from caafe import data
import pandas as pd
import torch
import heapq
from sklearn.neural_network import MLPClassifier, MLPRegressor
import statistics
from sklearn.metrics import roc_auc_score

# Load data observations
label_encoder = preprocessing.LabelEncoder()
pb_name = ['breast-w', 'crab']

feature_fn = []
seed = 42
for problem_name in pb_name:
    is_regression =False
    if problem_name in ['forest-fires', 'housing', 'insurance', 'bike', 'wine', 'crab']:
        is_regression = True
    llm= "gpt3.5"
    # Load data observations
    file_name = f"./data/{problem_name}.csv"

    df = pd.read_csv(file_name)
    target_attr = df.columns[-1]
    
    is_cat = [is_categorical(df.iloc[:, i]) for i in range(df.shape[1])][:-1]
    attribute_names = df.columns[:-1].tolist()

    X = df.convert_dtypes()
    y = df[target_attr].to_numpy()
    label_list = np.unique(y).tolist()

    X = X.drop(target_attr, axis=1)
    for col in X.columns:
        if X[col].dtype == 'string':
            X[col] = label_encoder.fit_transform(X[col])
    if is_regression == False:
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    else:
        skf = KFold(n_splits=5, shuffle=True, random_state=seed)
    i = 1
    max_score_avg = []
    for train_idx, test_idx in skf.split(X, y):
        test_outputs_all = []
        max_score = -1000000.0
        label_encoder = preprocessing.LabelEncoder()

        X_train_fold, X_test_fold = X.iloc[train_idx], X.iloc[test_idx]
        y_train_fold, y_test_fold = y[train_idx], y[test_idx]
        directory = f'./logs/{problem_name}_{llm}_split_{i}/samples'

        data = []

        for idx, filename in enumerate(os.listdir(directory)):
            if filename.endswith(".json"):
                filepath = os.path.join(directory, filename)
                with open(filepath, "r") as f:
                    if idx == 1:
                        data.append(max_score)                        
                    data.append(json.load(f)['score'])

        data = [max_score if v is None else v for v in data]
        best_k = heapq.nlargest(3, range(len(data)), key=data.__getitem__)
        for filename in best_k:
            if filename == 1:
                continue
            filepath = f'{directory}/samples_{filename}.json'
            with open(filepath, "r") as f:
                feature_fn = json.load(f)['function']
            exec(feature_fn)
            try:
                df_train_modified = modify_features(X_train_fold)
                for col in df_train_modified.columns:
                    if df_train_modified[col].dtype == 'string':
                        df_train_modified[col] = label_encoder.fit_transform(df_train_modified[col])
                
                df_test_modified = modify_features(X_test_fold)
                for col in df_test_modified.columns: 
                    if df_test_modified[col].dtype == 'string':
                        df_test_modified[col] = label_encoder.fit_transform(df_test_modified[col])
                df_train_new, df_test_new = make_datasets_numeric(df_train_modified, df_test_modified, target_attr)

                X_train = torch.tensor(df_train_new.to_numpy())
                X_test = torch.tensor(df_test_new.to_numpy())
                
                if is_regression == True:
                    model = xgb.XGBRegressor(random_state=42)
                    y_train = y_train_fold
                    y_test = y_test_fold
                    X_train_new = X_train
                    X_test_new = X_test
                else:
                    model = xgb.XGBClassifier(random_state=42)
                    label_encoder = preprocessing.LabelEncoder()
                    y_train = label_encoder.fit_transform(y_train_fold)
                    y_test = label_encoder.fit_transform(y_test_fold)
                    X_train_new = X_train
                    X_test_new = X_test

                # After fitting the model:
                model.fit(torch.nan_to_num(X_train_new), y_train)

                # Get predictions from the model.
                if is_regression:
                    # For regression, predict target values directly.
                    y_pred = model.predict(torch.nan_to_num(X_test_new))
                else:
                    # For classification, get class probabilities.
                    y_pred = model.predict(torch.nan_to_num(X_test_new))
                    
                test_outputs_all.append(y_pred)
            except:
                continue

        if len(test_outputs_all) > 0:
            test_outputs_all = np.stack(test_outputs_all, axis=0)  # Shape: (n_candidates, n_samples, n_classes) for classification

        if is_regression:
            # For regression, average predictions and compute negative RMSE.
            pred = np.mean(test_outputs_all, axis=0)
            score = -mean_squared_error(y_test, pred, squared=False)
        else:
            # For classification, average the probability estimates.
            # Convert averaged probabilities to predicted class labels.
            from scipy import stats
            pred = stats.mode(test_outputs_all, axis=0)[0]

            score = accuracy_score(y_test, pred)

        max_score_avg.append(score)
        i +=1
    mean_val = sum(max_score_avg)/ len(max_score_avg)
    mean_val
    std_dev = statistics.stdev(max_score_avg)
    print(f'{problem_name} : {mean_val:.3f} +- {std_dev:.3f}')

balance-scale : 0.934 +- 0.046


/home/grads/nikhilsa/miniconda3/envs/tab_ml/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/grads/nikhilsa/miniconda3/envs/tab_ml/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/grads/nikhilsa/miniconda3/envs/tab_ml/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/grads/nikhilsa/miniconda3/envs/tab_ml/lib/python3.11/site-packages/caafe/preprocessing.py:44: FutureWarning: Downca

crab : -2.277 +- 0.129


/home/grads/nikhilsa/miniconda3/envs/tab_ml/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
